Data for this project: https://www.kaggle.com/c/fake-news/data?select=train.csv

Import Dependencies

In [4]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [5]:
#stopwords like my, this, that, etc. Remove this words from the news.
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Importing Dataset

In [7]:
data = pd.read_csv('/content/train.csv')
data.shape

(20800, 5)

In [8]:
#print first five rows of the dataset
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
#counting the missing value in dataset
data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [10]:
#Replacing null values with empty string
data = data.fillna('')

In [12]:
#merging the title name and news title and make a new column named content
#We will use this data to make the prediction
data['content'] = data['author']+data['title']
print(data['content'])

0        Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1        Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom...
2        Consortiumnews.comWhy the Truth Might Get You ...
3        Jessica Purkiss15 Civilians Killed In Single U...
4        Howard PortnoyIranian woman jailed for fiction...
                               ...                        
20795    Jerome HudsonRapper T.I.: Trump a ’Poster Chil...
20796    Benjamin HoffmanN.F.L. Playoffs: Schedule, Mat...
20797    Michael J. de la Merced and Rachel AbramsMacy’...
20798    Alex AnsaryNATO, Russia To Hold Parallel Exerc...
20799               David SwansonWhat Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [14]:
#seprating data and label column
X=data.drop(columns='label', axis=1)
Y=data['label']


Stemming  is the process of reducing a word to its Root Word

In [15]:
port_strm = PorterStemmer()
def stemming(content):
  strm_content = re.sub('[^a-zA-Z]',' ', content)
  strm_content = strm_content.lower()
  strm_content = strm_content.split()
  strm_content = [port_strm.stem(word) for word in strm_content if not word in stopwords.words('english')]
  strm_content = ' '.join(strm_content)
  return strm_content

In [16]:
data['content'] = data['content'].apply(stemming)

In [17]:
print(data['content'])

0        darrel lucushous dem aid even see comey letter...
1        daniel j flynnflynn hillari clinton big woman ...
2                consortiumnew comwhi truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoyiranian woman jail fiction unpub...
                               ...                        
20795    jerom hudsonrapp trump poster child white supr...
20796    benjamin hoffmann f l playoff schedul matchup ...
20797    michael j de la merc rachel abramsmaci said re...
20798    alex ansarynato russia hold parallel exercis b...
20799                        david swansonwhat keep f aliv
Name: content, Length: 20800, dtype: object


In [18]:
#Actual data to use for traning
X = data['content'].values
Y = data['label'].values

In [19]:
#converting text data to numerical data
vact = TfidfVectorizer()
vact.fit(X)
X = vact.transform(X)

In [20]:
print(X)

  (0, 26340)	0.28088379401596425
  (0, 22724)	0.2552336018069161
  (0, 15019)	0.43006226759639316
  (0, 14555)	0.29177259684200296
  (0, 12782)	0.24619727512767195
  (0, 8022)	0.23133661742488731
  (0, 6273)	0.2839932825877813
  (0, 5969)	0.35488202138141456
  (0, 5006)	0.2472595823572816
  (0, 4211)	0.3625320323150658
  (0, 578)	0.2694167078545385
  (1, 27923)	0.36911845953845024
  (1, 11313)	0.24166773097712638
  (1, 8772)	0.5258635625386451
  (1, 5916)	0.31810058109638056
  (1, 4767)	0.23338756776626793
  (1, 3859)	0.45980466668763476
  (1, 3281)	0.18652439327549428
  (1, 2622)	0.3562953366945267
  (2, 26235)	0.3665032495181434
  (2, 16361)	0.43295215406038445
  (2, 9454)	0.30743020569262086
  (2, 8567)	0.3411947414020896
  (2, 5240)	0.40440534260277944
  (2, 5121)	0.5511414848555652
  :	:
  (20797, 25776)	0.08220218573989037
  (20797, 25319)	0.3119640221826561
  (20797, 22086)	0.24902354987792552
  (20797, 20778)	0.2729578683228216
  (20797, 20493)	0.249994989010826
  (20797, 17505

Spliting data to train and test data

In [21]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, stratify=Y, random_state=2)

Training the Model: Logistic Regression

In [22]:
model = LogisticRegression()
model.fit(X_train, Y_train)

LogisticRegression()

Model Evoluation

Accuracy Score on the Training Data

In [24]:
prediction = model.predict(X_train)
accuracy = accuracy_score(prediction, Y_train)
print('Accuracy Score on the Training Data: ', accuracy)

Accuracy Score on the Training Data:  0.9719951923076923


Accuracy Score on the Test Data

In [25]:
prediction = model.predict(X_test)
accuracy = accuracy_score(prediction, Y_test)
print('Accuracy Score on the Test Data: ', accuracy)

Accuracy Score on the Test Data:  0.9548076923076924


Making Predictive System

In [29]:
X_news = X_test[10]
predic = model.predict(X_news)
print(predic)

if(predic[0]==0):
  print('News is Real')
else:
  print('News is Fake')

[0]
News is Real


In [30]:
print(Y_test[10])

0
